In [ ]:
import requests
import json
import torch
import os
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## connect google drive in order to save the model

In [ ]:
if not os.path.exists('/content/drive/MyDrive/BERT-SQuAD'):
  os.mkdir('/content/drive/MyDrive/BERT-SQuAD')


In [ ]:
!pip install transformers

## Download Squad 2.0

In [ ]:
#loading training dataset and take a look at it
with open('/content/drive/MyDrive/chatbot/bertN_B_train.json','rb') as f:
  squad = json.load(f)

In [ ]:
#each 'data has two keys(title and paragraphs)
squad['data'][0].keys()

dict_keys(['title', 'paragraphs'])

After we got a taste of the jsons files data format let's extract our data and store them into some data structures.

In [ ]:
def read_data(path):
  # load the json file
  with open(path, 'rb') as f:
    squad = json.load(f)

  contexts = []
  questions = []
  answers = []

  for group in squad['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts, questions, answers

Put the contexts, questions and answers for training and validation into the appropriate lists.

In [ ]:
train_contexts, train_questions, train_answers = read_data('/content/drive/MyDrive/chatbot/bertN_B_train.json')
valid_contexts, valid_questions, valid_answers = read_data('/content/drive/MyDrive/chatbot/bertN_B_test.json')

In [ ]:
# print a random question and answer
print(f'There are {len(valid_questions)} questions')
print(valid_questions[2])
print(valid_contexts[2])

There are 65 questions
Can you pinpoint where exactly the Bhairavanath Temple is situated within Bhaktapur?
Bhairavanath temple is one of the temples that lies at the central of Bhaktapur, Taumadhii Square and Bhairavanath Temple is built in the pagoda style and It is three-storey. It is one of the temple devoted to Lord Bhairava is one of the rectangular based temples of Bhaktapur. Curiously, there is no entrance on the temple's lengthy west facade that faces the square. Instead, priests access the interior from the south. The west facade is largely symmetrical and features a ground-level shrine with Bhairav depicted in miniature on a small metal throne. Michael Hutt notes that this small statue is frequently stolen and replaced. To either side of the deity are hammered copper plates depicting Bhairav's eyes, two pairs of hands, and two dogs and his faithful companions. At present, perhaps to discourage theft, the ground-level shrine is blocked from casual access by a makeshift wooden

As you can see above, the answers are dictionaries whith the answer text and an integer which indicates the start index of the answer in the context. As the SQuAD does not give us the end index of the answer in the context we have to find it ourselves. So, let's get the character position at which the answer ends in the passage. Note that sometimes SQuAD answers are off by one or two characters, so we will also adjust for that.

In [ ]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(valid_answers, valid_contexts)

In [ ]:
# You can see that now we get the answer_end also
print(valid_questions[40])
print(valid_answers[40])
print(valid_contexts)

What do those etched stones represent?
{'text': 'The lowest are statues of Jayalla Malla and Patta. Two champions of Bhatgaon Raja, which of whom is said to have had the strength of ten men, The next are elephants, ten times as strong as the men, The third are lions, ten times as strong as the elephants, The fourth are sarduls or griffins, ten times as strong as the lions, And the fifth are stone Byaghrini, two goddesses of supernatural power.', 'answer_start': 4461, 'answer_end': 4859}
["Bhairavanath temple is one of the temples that lies at the central of Bhaktapur, Taumadhii Square and Bhairavanath Temple is built in the pagoda style and It is three-storey. It is one of the temple devoted to Lord Bhairava is one of the rectangular based temples of Bhaktapur. Curiously, there is no entrance on the temple's lengthy west facade that faces the square. Instead, priests access the interior from the south. The west facade is largely symmetrical and features a ground-level shrine with Bhair

### Tokenization
As we know we have to tokenize our data in form that is acceptable for the BERT model. We are going to use the BertTokenizerFast instead of BertTokenizer as the first one is much faster. Since we are going to train our model in batches we need to set padding=True

Models based on the transformers architecture have become a state-of-the-art solution in NLP. The word “transformer” is indeed what the letter “T” stands for in the names of the famous BERT, GPT3 and the massively popular nowadays ChatGPT. The common obstacle while applying these models is the constraint on the input length. For example, the BERT model cannot process texts which are longer than 512 tokens (roughly speaking, one token is associated with one word).

The method to overcome this issue was proposed by Devlin (one of the authors of BERT) in the discussion. In this article, we will describe in detail how to modify the process of fine-tuning a pre-trained BERT model for the classification task. The code is available as open source here.

In [ ]:
from transformers import BertTokenizerFast
max_length = 1536

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer.model_max_length = max_length


train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True)

In [ ]:
#tokens = tokenizer(train_contexts, add_special_tokens=False, truncation=False, return_tensors="pt")

SyntaxError: ignored

Let's see what we got after tokenizing our data.



In [ ]:
#train_encodings["input_ids"]

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 303 context-question pairs


In [ ]:
train_encodings['input_ids'][0]


[101,
 1038,
 26227,
 12462,
 16207,
 3379,
 2003,
 2028,
 1997,
 1996,
 8436,
 2008,
 3658,
 2012,
 1996,
 2430,
 1997,
 1038,
 20459,
 2696,
 5311,
 1010,
 19982,
 25666,
 4048,
 2072,
 2675,
 1998,
 1038,
 26227,
 12462,
 16207,
 3379,
 2003,
 2328,
 1999,
 1996,
 27387,
 2806,
 1998,
 2009,
 2003,
 2093,
 1011,
 11676,
 1012,
 2009,
 2003,
 2028,
 1997,
 1996,
 3379,
 7422,
 2000,
 2935,
 1038,
 26227,
 12462,
 2003,
 2028,
 1997,
 1996,
 10806,
 2241,
 8436,
 1997,
 1038,
 20459,
 2696,
 5311,
 1012,
 16484,
 1010,
 2045,
 2003,
 2053,
 4211,
 2006,
 1996,
 3379,
 1005,
 1055,
 12401,
 2225,
 8508,
 2008,
 5344,
 1996,
 2675,
 1012,
 2612,
 1010,
 8656,
 3229,
 1996,
 4592,
 2013,
 1996,
 2148,
 1012,
 1996,
 2225,
 8508,
 2003,
 4321,
 23476,
 1998,
 2838,
 1037,
 2598,
 1011,
 2504,
 9571,
 2007,
 1038,
 26227,
 11431,
 8212,
 1999,
 12973,
 2006,
 1037,
 2235,
 3384,
 6106,
 1012,
 2745,
 12570,
 2102,
 3964,
 2008,
 2023,
 2235,
 6231,
 2003,
 4703,
 7376,
 1998,
 2999,
 1012,

Let's decode the first pair of context-question encoded pair and look into it.



In [ ]:
tokenizer.decode(train_encodings['input_ids'][0])


"[CLS] bhairavanath temple is one of the temples that lies at the central of bhaktapur, taumadhii square and bhairavanath temple is built in the pagoda style and it is three - storey. it is one of the temple devoted to lord bhairava is one of the rectangular based temples of bhaktapur. curiously, there is no entrance on the temple's lengthy west facade that faces the square. instead, priests access the interior from the south. the west facade is largely symmetrical and features a ground - level shrine with bhairav depicted in miniature on a small metal throne. michael hutt notes that this small statue is frequently stolen and replaced. to either side of the deity are hammered copper plates depicting bhairav's eyes, two pairs of hands, and two dogs and his faithful companions. at present, perhaps to discourage theft, the ground - level shrine is blocked from casual access by a makeshift wooden barricade. the temple is dedicated to lord bhairav, which is supposed to be the most ferocious

Next we need to convert our character start/end positions to token start/end positions. Why is that? Because our words converted into tokens, so the answer start/end needs to show the index of start/end token which contains the answer and not the specific characters in the context.



In [ ]:
# List to store indices where 'answer_end' is missing
missing_indices = []

# Loop through train_answers to find dictionaries without 'answer_end'
for idx, answer in enumerate(valid_answers):
    if 'answer_end' not in answer:
        missing_indices.append(idx)

# Display indices where 'answer_end' is missing
if len(missing_indices) > 0:
    print("Indices where 'answer_end' is missing:")
    print(missing_indices)
else:
    print("All dictionaries have 'answer_end' key.")

All dictionaries have 'answer_end' key.


In [ ]:
# import json
# import os

# # Define the directory path where you want to save the file
# directory_path = '/content/drive/MyDrive/chatbot/testingfiles/'

# # Ensure the directory exists or create it if it doesn't
# if not os.path.exists(directory_path):
#     os.makedirs(directory_path)

# # Specify the file path within the directory
# file_path = os.path.join(directory_path, 'valid_answers.json')

# # Save train_answers to the specified file path as a JSON file
# with open(file_path, 'w') as file:
#     json.dump(valid_answers, file)

In [ ]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(valid_encodings, valid_answers)

In [ ]:

train_encodings['start_positions'][:10]

[1, 1, 1, 1, 1, 28, 28, 28, 28, 28]

### Dataset defination
We have to define our dataset using the PyTorch Dataset class from torch.utils in order create our dataloaders after that.

In [ ]:
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = SQuAD_Dataset(train_encodings)
valid_dataset = SQuAD_Dataset(valid_encodings)

### Dataloaders

In [ ]:
from torch.utils.data import DataLoader

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)

### Fine-tuning

### model defination

We are going to use the bert-case-uncased from the huggingface transformers

In [ ]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")
model.config.max_position_embeddings = max_length
model.base_model.embeddings.position_ids = torch.arange(max_length).expand((1, -1))
model.base_model.embeddings.token_type_ids = torch.zeros(max_length).expand((1, -1))
orig_pos_emb = model.base_model.embeddings.position_embeddings.weight
model.base_model.embeddings.position_embeddings.weight = torch.nn.Parameter(torch.cat((orig_pos_emb, orig_pos_emb)))

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training
Μy choices for some parameters:

Use of AdamW which is a stochastic optimization method that modifies the typical implementation of weight decay in Adam, by decoupling weight decay from the gradient update. This helps to avoid overfitting which is necessary in this case were the model is very complex.

Set the lr=5e-5 as I read that this is the best value for the learning rate for this task.

In [ ]:
# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [ ]:
input_ids = input_ids.to(torch.long) # Convert to Long type and move to CPU



In [ ]:
from transformers import AdamW

N_EPOCHS = 5
optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

  0%|          | 0/19 [00:00<?, ?it/s]


RuntimeError: ignored

## save the model in my drive


In [ ]:
model_path = '/content/drive/MyDrive/BERT-SQuAD'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/BERT-SQuAD/tokenizer_config.json',
 '/content/drive/MyDrive/BERT-SQuAD/special_tokens_map.json',
 '/content/drive/MyDrive/BERT-SQuAD/vocab.txt',
 '/content/drive/MyDrive/BERT-SQuAD/added_tokens.json',
 '/content/drive/MyDrive/BERT-SQuAD/tokenizer.json')

Respectively load the saved model

In [ ]:

from transformers import BertForQuestionAnswering, BertTokenizerFast

model_path = '/content/drive/MyDrive/BERT-SQuAD'
model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

model = model.to(device)


Working on cuda


### Testing
We are evaluating the model on the validation set by checking the model's predictions for the answer's start and end indexes and comparing with the true ones.



In [ ]:
model.eval()

acc = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

acc = sum(acc)/len(acc)

print("\n\nT/P\tanswer_start\tanswer_end\n")
for i in range(len(start_true)):
  print(f"true\t{start_true[i]}\t{end_true[i]}\n"
        f"pred\t{start_pred[i]}\t{end_pred[i]}\n")

100%|██████████| 5/5 [00:02<00:00,  2.27it/s]



T/P	answer_start	answer_end

true	114	125
pred	87	191



### Ask
We are going to use some functions from the official Evaluation Script v2.0 of SQuAD in order to test the fine-tuned model by asking some questions given a context. I have also looked at this notebook which evaluates BERT on SQuAD.



In [ ]:
def get_prediction(context, question):
  inputs = tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
  outputs = model(**inputs)

  answer_start = torch.argmax(outputs[0])
  answer_end = torch.argmax(outputs[1]) + 1

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

this is the google wala code

In [ ]:
# import torch

# def get_prediction(context, question):
#     inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

#     # Move inputs to the same device as the model
#     inputs = {key: tensor.to(model.device) for key, tensor in inputs.items()}

#     with torch.no_grad():
#         outputs = model(**inputs)

#     start_scores = outputs.start_logits
#     end_scores = outputs.end_logits

#     # Get the most likely answer using the start and end scores
#     all_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())
#     answer_tokens = all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
#     answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

#     return answer

In [ ]:
# # Example context
# context = "Bhairavanath temple is one of the temples that lies at the central of Bhaktapur, Taumadhii Square and Bhairavanath Temple is built in the pagoda style and It is three-storey. It is one of the temple devoted to Lord Bhairava is one of the rectangular based temples of Bhaktapur. Curiously, there is no entrance on the temple's lengthy west facade that faces the square. Instead, priests access the interior from the south. The west facade is largely symmetrical and features a ground-level shrine with Bhairav depicted in miniature on a small metal throne. Michael Hutt notes that this small statue is frequently stolen and replaced. To either side of the deity are hammered copper plates depicting Bhairav's eyes, two pairs of hands, and two dogs and his faithful companions. At present, perhaps to discourage theft, the ground-level shrine is blocked from casual access by a makeshift wooden barricade. The temple is dedicated to lord Bhairav, which is supposed to be the most ferocious form of Lord Shiva. The Bhairavanath established in this temple is a formidable incarnation of lord shiva, who also prefers to name Kasi Vishwanath and Aakash bhairava. He is somehow famous for his obstreperous and stubborn character. Bhairav has a particular fascination with machines and is often thought to inhabit wheels, with different versions of the god inhabiting different wheels on the same chariot. At the Bhairavanath temple, a chariot used in the annual Bisket-Jatra festival is stored, dismantled, against the temple's north wall. Bhairavanath's association with the chariot used in the festival highlights his connection to the annual Bisket-Jatra festival. The statue of Lord Bhairav is about 12 feet long and is one of the largest statues in Kathmandu Valley. Thousands of people visit Bhairavanath temple during Bhairav Ashtami or Bhairav Jayanthi as it is celebrated religiously.  Even though the temple was slightly damaged during the earthquake in November 2015, it still has retrained most of structure. The Renovative history of Bhairavanath Temple says that The temple was first inaugurated by King Ananda Deva in the 13th century."

# # Ask a question
# question = "Where is Bhairavanath located?"
# answer = get_prediction(context, question)
# print("Answer:", answer)

Answer: 


In [ ]:
def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re
  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)
  def white_space_fix(text):
    return " ".join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))


In [ ]:
def exact_match(prediction, truth):
    return bool(normalize_text(prediction) == normalize_text(truth))


In [ ]:
def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return round(2 * (prec * rec) / (prec + rec), 2)

In [ ]:
def question_answer(context, question,answer):
  prediction = get_prediction(context,question)
  em_score = exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f'Question: {question}')
  print(f'Prediction: {prediction}')
  print(f'True Answer: {answer}')
  print(f'Exact match: {em_score}')
  print(f'F1 score: {f1_score}\n')

### Bhairavanath

In [ ]:
context = """Bhairavanath temple is one of the temples that lies at the central of Bhaktapur, Taumadhii Square and Bhairavanath Temple is built in the pagoda style and It is three-storey. It is one of the temple devoted to Lord Bhairava is one of the rectangular based temples of Bhaktapur. Curiously, there is no entrance on the temple's lengthy west facade that faces the square. Instead, priests access the interior from the south. The west facade is largely symmetrical and features a ground-level shrine with Bhairav depicted in miniature on a small metal throne. Michael Hutt notes that this small statue is frequently stolen and replaced. To either side of the deity are hammered copper plates depicting Bhairav's eyes, two pairs of hands, and two dogs and his faithful companions. At present, perhaps to discourage theft, the ground-level shrine is blocked from casual access by a makeshift wooden barricade. The temple is dedicated to lord Bhairav, which is supposed to be the most ferocious form of Lord Shiva. The Bhairavanath established in this temple is a formidable incarnation of lord shiva, who also prefers to name Kasi Vishwanath and Aakash bhairava. He is somehow famous for his obstreperous and stubborn character. Bhairav has a particular fascination with machines and is often thought to inhabit wheels, with different versions of the god inhabiting different wheels on the same chariot. At the Bhairavanath temple, a chariot used in the annual Bisket-Jatra festival is stored, dismantled, against the temple's north wall. Bhairavanath's association with the chariot used in the festival highlights his connection to the annual Bisket-Jatra festival. The statue of Lord Bhairav is about 12 feet long and is one of the largest statues in Kathmandu Valley. Thousands of people visit Bhairavanath temple during Bhairav Ashtami or Bhairav Jayanthi as it is celebrated religiously.  Even though the temple was slightly damaged during the earthquake in November 2015, it still has retrained most of structure. The Renovative history of Bhairavanath Temple says that The temple was first inaugurated by King Ananda Deva in the 13th century."""
questions=["Where is Bhairavanath located?",
           "Tell me the legends about the temple?",
           "what deity is the temple dedicated to?"]
answers=["Bhairavanath temple is one of the temples that lies at the central of Bhaktapur, Taumadhii Square",
         "There are legends about the temple. The inner part of the temple consists of the head of Lord Bhairav. There are many stories as to why and how Lord Bhairav\u2019s head was cut off and worshipped here. According to legend, long ago, Kasi Bhairabnath  traveled to Nepal to view either the Bisket Jatra  or the Bal Kaumari Jatra. A tantric priestess in the crowd recognized him for what he was and attempted to immobilize him, but Vishvanath sought to escape by sinking into the ground. As his body descended into the earth, the priestess realized the god was her own husband. To prevent his escape, she ordered his head cut off where it protruded from the ground. Being a god, this was far from a fatal wound, and the priestess enshrined the head in the temple that now stands here. The body, meanwhile, returned to Benares whence it came. Dowman notes that a headless image of Bhairav is still worshipped there even today. Another story about this temple is also interesting, passed orally by our ancestors. According to this, the lord bhairava of Varanasi once visited Bhaktapur to see Bisket jatra. Bisket jatra is one of the biggest jatras of the newar people. During the festival, the lord Bhairava somehow was recognized by a priest and he just cast a decoy spell on him.  As a quick reaction, he started to sink into the terra firma to go back to Varanasi. But the priest didnot let him go. As if, he grabs the head of Bhairava in his hand and decollates him with his sword. That is why people say that the head doesnot match the body of Kasi Vishwanath. It is also believed that the head is still treasured in a box of this temple.",
         "The temple is dedicated to lord Bhairav, which is supposed to be the most ferocious form of Lord Shiva"]
for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

Question: Where is Bhairavanath located?
Prediction: 
True Answer: Bhairavanath temple is one of the temples that lies at the central of Bhaktapur, Taumadhii Square
Exact match: False
F1 score: 0

Question: Tell me the legends about the temple?
Prediction: 
True Answer: There are legends about the temple. The inner part of the temple consists of the head of Lord Bhairav. There are many stories as to why and how Lord Bhairav’s head was cut off and worshipped here. According to legend, long ago, Kasi Bhairabnath  traveled to Nepal to view either the Bisket Jatra  or the Bal Kaumari Jatra. A tantric priestess in the crowd recognized him for what he was and attempted to immobilize him, but Vishvanath sought to escape by sinking into the ground. As his body descended into the earth, the priestess realized the god was her own husband. To prevent his escape, she ordered his head cut off where it protruded from the ground. Being a god, this was far from a fatal wound, and the priestess enshrin

In [ ]:
context = """Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer,
          songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing
          and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child.
          Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time.
          Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide,
          earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy"."""


questions = ["For whom the passage is talking about?",
             "When did Beyonce born?",
             "Where did Beyonce born?",
             "What is Beyonce's nationality?",
             "Who was the Destiny's group manager?",
             "What name has the Beyoncé's debut album?",
             "How many Grammy Awards did Beyonce earn?",
             "When did the Beyoncé's debut album release?",
             "Who was the lead singer of R&B girl-group Destiny's Child?"]

answers = ["Beyonce Giselle Knowles - Carter", "September 4, 1981", "Houston, Texas",
           "American", "Mathew Knowles", "Dangerously in Love", "five", "2003",
           "Beyonce Giselle Knowles - Carter"]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

Question: For whom the passage is talking about?
Prediction: the group became one of the world ' s best - selling girl groups of all time. their hiatus saw the release of beyonce ' s debut album, dangerously in love ( 2003 ), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number - one singles " crazy in love " and " baby boy ".
True Answer: Beyonce Giselle Knowles - Carter
Exact match: False
F1 score: 0.04

Question: When did Beyonce born?
Prediction: the group became one of the world ' s best - selling girl groups of all time. their hiatus saw the release of beyonce ' s debut album, dangerously in love ( 2003 ), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number - one singles " crazy in love " and " baby boy ".
True Answer: September 4, 1981
Exact match: False
F1 score: 0

Question: Where did Beyonce born?
Prediction: the group became one of the world ' s 

In [ ]:
context = """Athens is the capital and largest city of Greece. Athens dominates the Attica region and is one of the world's oldest cities,
             with its recorded history spanning over 3,400 years and its earliest human presence starting somewhere between the 11th and 7th millennium BC.
             Classical Athens was a powerful city-state. It was a center for the arts, learning and philosophy, and the home of Plato's Academy and Aristotle's Lyceum.
             It is widely referred to as the cradle of Western civilization and the birthplace of democracy, largely because of its cultural and political impact on the European continent—particularly Ancient Rome.
             In modern times, Athens is a large cosmopolitan metropolis and central to economic, financial, industrial, maritime, political and cultural life in Greece.
             In 2021, Athens' urban area hosted more than three and a half million people, which is around 35% of the entire population of Greece.
             Athens is a Beta global city according to the Globalization and World Cities Research Network, and is one of the biggest economic centers in Southeastern Europe.
             It also has a large financial sector, and its port Piraeus is both the largest passenger port in Europe, and the second largest in the world."""

questions = ["Which is the largest city in Greece?",
             "For what was the Athens center?",
             "Which city was the home of Plato's Academy?"]

answers = ["Athens", "center for the arts, learning and philosophy", "Athens"]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

Question: Which is the largest city in Greece?
Prediction: in modern times, athens is a large cosmopolitan metropolis and central to economic, financial, industrial, maritime, political and cultural life in greece. in 2021, athens ' urban area hosted more than three and a half million people, which is around 35 % of the entire population of greece.
True Answer: Athens
Exact match: False
F1 score: 0.05

Question: For what was the Athens center?
Prediction: in modern times, athens is a large cosmopolitan metropolis and central to economic, financial, industrial, maritime, political and cultural life in greece. in 2021, athens ' urban area hosted more than three and a half million people, which is around 35 % of the entire population of greece.
True Answer: center for the arts, learning and philosophy
Exact match: False
F1 score: 0.04

Question: Which city was the home of Plato's Academy?
Prediction: in modern times, athens is a large cosmopolitan metropolis and central to economic, finan